In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys #need to send keystrokes
from datetime import datetime 
from datetime import timedelta
from dateutil import parser
import urllib
import csv
import time

In [5]:
# create new instance of chrome 
browser = webdriver.Chrome(executable_path='C:/webdriver/chromedriver')

start_date = '2019-02-21'
start_date = parser.parse(start_date)

SCROLL_PAUSE_TIME = 0.5


In [ ]:
with open('AirBnbOneDay.csv','w',encoding="utf-8",newline='') as csvfile:
    fieldnames = ['Date','Title','Price','Guests','Review','Review_stars']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    # how many days to loop through
    for h in range(1,4):
        Checkin_Date = start_date+timedelta(days=1)# check_in =19+1=20
        Checkout_Date = start_date+timedelta(days=2)# check_out = 19+2=21
        browser.get("https://www.airbnb.ca/s/homes?refinement_paths%5B%5D=%2Fhomes&query=New%20York&allow_override%5B%5D=&checkin="+Checkin_Date.strftime('%Y-%m-%d')+"&checkout="+Checkout_Date.strftime('%Y-%m-%d')+"&map_toggle=false")
        browser.maximize_window()
        last_height = browser.execute_script("return document.body.scrollHeight")# Get scroll height
        then = time.time()
        while True:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to bottom
            time.sleep(SCROLL_PAUSE_TIME) # Wait to load page
            new_height = browser.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
            if new_height == last_height or time.time()-then > 60:
                break
        last_height = new_height
        cont = browser.find_element_by_xpath("//body[@class='with-new-header has-epcot-header']")
        prods = cont.find_elements_by_xpath("//meta[@itemprop='url']")
        links = [prod.get_attribute("content") for prod in prods]  
        for l in links: # 18 or 30 based on the num of the homes in the pag
            driver = webdriver.Chrome(executable_path='C:/webdriver/chromedriver')
            driver.get("https://"+l)
            tot = driver.find_element_by_xpath("//body[@class='with-new-header has-epcot-header']")


            try:
                title = tot.find_element_by_class_name('_18hrqvin').text
            except NoSuchElementException:
                title = None

            try:
                price = tot.find_element_by_class_name('_doc79r').text
            except NoSuchElementException:
                price = None

            try:
                review_stars = tot.find_element_by_xpath("//button[@class='_ff6jfq']").get_attribute("aria-label")
            except NoSuchElementException:
                review_stars = None

            try:
                num_guests = tot.find_element_by_xpath("//span[@class='_1p3joamp']").text
            except NoSuchElementException:
                num_guests = None

            try:
                review_txt = tot.find_element_by_xpath("//div[@id='reviews']").text
            except NoSuchElementException:
                review_txt = None

            writer.writerow({'Date' : Checkin_Date,
                             'Title': title,
                             'Price': price,
                             'Guests': num_guests,
                             'Review_stars': review_stars,
                             'Review':review_txt})

            time.sleep(10)
            driver.close()
            
        #start_date = Checkin_Date
                
                
                
                
               # browser.find_element_by_xpath("//li[@class='_b8vexar']/a").send_keys("\n") #send enter for links, buttons
            #inputElement.send_keys("\n") #send enter for links, buttons
         # Calculate new scroll height and compare with last scroll height
        